# BDA 8기의 학습 데이터를 분석하여 9기 학습자의 수료 여부를 예측하는 AI 알고리즘을 개발하기

- 기수(전부 8기), 자연어인 열 삭제 (columns_to_exclude)
- 결측치 50% 이상인 열 삭제
- lightGBM

In [56]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [57]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 엑셀 파일 불러오기
df = pd.read_csv("train.csv", encoding="utf-8")


In [58]:
df.head()

,ID,generation,school1,major type,major1_1,major1_2,major_data,job,class1,class2,class3,class4,re_registration,contest_award,nationality,inflow_route,whyBDA,what_to_gain,hope_for_group,previous_class_3,previous_class_4,previous_class_5,previous_class_6,previous_class_7,major_field,desired_career_path,completed_semester,project_type,time_input,desired_job,certificate_acquisition,desired_certificate,certificate_study_period,desired_job_except_data,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,incumbents_lecture_scale_reason,interested_company,expected_domain,contest_participitation,idea_contest,onedayclass_topic,withdrawal
0,TRAIN_0000,8,3e0057be3ecde495d1d6aab0bdc3092cce94440a6a4f68...,단일 전공,IT(컴퓨터 공학 포함),NaN,False,대학생,4,NaN,NaN,NaN,아니요,NaN,내국인,에브리타임,"BDA 학회원만의 혜택을 누리고 싶어서(현직자 강연, 잡 페스티벌, 기업연계 공모전 등)",프로젝트 경험,네. 오프라인으로 참여하고 싶어요,NaN,NaN,NaN,NaN,NaN,공학 (컴퓨터 공학 제외),취업,대학교 이수학기 4학기 이하,팀,4.5,데이터 엔지니어,없음,"SQLD, 정보처리기사",시험일 전 4주 (2/1 ~ 2/27),개발,주니어 (0~3년차),커리어 패스 과정 (예시: 비전공자/전공자의 취업 준비 및 이직 과정),"국내 대기업 IT 계열 (금융, 제조 ...)","온,오프라인 동시",3~50명 내외의 강의 리스너와 1명의 현직자,인원이 너무 많지 않아야 편하게 들을 수 있어서,삼성이나 하이닉스,AI,NaN,NaN,C++ 응용,1
1,TRAIN_0001,8,c7c76d1a4c9d719d17c899a3ca9de153452a22163fe083...,"복수 전공 ( 다중전공, 이중전공 포함 )",IT(컴퓨터 공학 포함),IT(컴퓨터 공학 포함),False,대학생,4,NaN,NaN,NaN,아니요,NaN,내국인,에브리타임,혼자 공부하기 어려워서,프로젝트 경험,네. 오프라인으로 참여하고 싶어요,NaN,NaN,NaN,NaN,NaN,IT (컴퓨터 공학 포함),취업,대학교 이수학기 5학기 이상,팀,2.0,인공지능 전문가,없음,"ADsP, SQLD, 빅데이터 분석 기사",시험일 전 4주 (2/1 ~ 2/27),AI,주니어 (0~3년차),직무 강의 (예시: 실무 진행 방식 및 직무 준비생을 위한 팁),국내 빅테크 IT 계열 (네카쿠라배당토),"온,오프라인 동시",10명 내외의 강의 리스너와 1명의 현직자,인원이 적어야 집중할 수 있고 질문 하기 쉽다.,네이버,AI,NaN,NaN,.,1
2,TRAIN_0002,8,1ed9296bd2a05ea36fabb5171a003b7320bb21a53d6135...,단일 전공,IT(컴퓨터 공학 포함),NaN,True,대학생,4,NaN,NaN,NaN,아니요,NaN,내국인,에브리타임,혼자 공부하기 어려워서,데이터 분석 역량,네. 오프라인으로 참여하고 싶어요,NaN,NaN,NaN,NaN,NaN,IT (컴퓨터 공학 포함),취업,대학교 이수학기 4학기 이하,팀,3.0,데이터 분석가,없음,빅데이터 분석 기사,시험일 전 4주 (2/1 ~ 2/27),AI,시니어 (10년차 ~),커리어 패스 과정 (예시: 비전공자/전공자의 취업 준비 및 이직 과정),"국내 대기업 IT 계열 (금융, 제조 ...)","온,오프라인 동시",3~50명 내외의 강의 리스너와 1명의 현직자,너무 소수인원은 조금 부담스럽고 너무 많은 인원은 집중도가 떨어질거같아서,스포츠분석 기업,AI,139ac02d03d83f839197195e65bc1810ef5282ade32d80...,NaN,C언어 기초,1
3,TRAIN_0003,8,5ec50ef5a0c0416652f8d67a89ef4c02eb0c6a61b5d0a8...,단일 전공,IT(컴퓨터 공학 포함),NaN,True,대학생,7,NaN,NaN,NaN,아니요,NaN,내국인,에브리타임,현직자의 강의를 듣고 싶어서,데이터 분석 역량,네. 온라인으로 참여하고 싶어요,NaN,NaN,NaN,NaN,NaN,자연과학,취업,대학교 이수학기 5학기 이상,팀,2.0,데이터 분석가,컴퓨터활용능력,"ADsP, SQLD, 빅데이터 분석 기사, 정보처리기사",시험일 전 4주 (2/1 ~ 2/27),AI,주니어 (0~3년차),커리어 패스 과정 (예시: 비전공자/전공자의 취업 준비 및 이직 과정),국내 빅테크 IT 계열 (네카쿠라배당토),"온,오프라인 동시",3~50명 내외의 강의 리스너와 1명의 현직자,너무 많은 인원이 있을 경우 집중도가 떨어질 거 같습니다.,토스,AI,NaN,NaN,현직자의 취업과정,0
4,TRAIN_0004,8,c7c76d1a4c9d719d17c899a3ca9de153452a22163fe083...,"복수 전공 ( 다중전공, 이중전공 포함 )",IT(컴퓨터 공학 포함),IT(컴퓨터 공학 포함),False,대학생,7,NaN,NaN,NaN,아니요,NaN,내국인,에브리타임,혼자 공부하기 어려워서,공모전 경험,네. 오프라인으로 참여하고 싶어요,NaN,NaN,NaN,NaN,NaN,"IT (컴퓨터 공학 포함), 인문학",취업,대학교 이수학기 5학기 이상,팀,1.5,데이터 분석가,SQLD,"ADsP, 빅데이터 분석 기사, 정보처리기사",시험일 전 4주 (2/1 ~ 2/27),개발,주니어 (0~3년차),커리어 패스 과정 (예시: 비전공자/전공자의 취업 준비 및 이직 과정),"국내 대기업 IT 계열 (금융, 제조 ...)",오프라인,100명 이상의 리스너와 10명 이상의 현직자,많은 현직자분들의 이야기를 듣고싶어서,네이버 카카오,금융,NaN,NaN,C언어,1


In [59]:
# 1056행의 데이터이므로, 1056개의 행을 가진 컬럼은 필수 응답 및 선택형 답변일 가능성이 높다고 생각했음
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056 entries, 0 to 1055
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               1056 non-null   object 
 1   generation                       1056 non-null   int64  
 2   school1                          1056 non-null   object 
 3   major type                       1049 non-null   object 
 4   major1_1                         1051 non-null   object 
 5   major1_2                         493 non-null    object 
 6   major_data                       1056 non-null   bool   
 7   job                              1056 non-null   object 
 8   class1                           1056 non-null   int64  
 9   class2                           101 non-null    float64
 10  class3                           2 non-null      float64
 11  class4                           1 non-null      float64
 12  re_registration     

In [60]:
columns_to_exclude = [

    "generation", # 전부 8임
    
    "incumbents_lecture",
    "desired_career_path",
    "desired_job",
    "incumbents_company_level",
    "incumbents_lecture_scale",
    "incumbents_lecture_scale_reason",
    "interested_company",
    "expected_domain",
    "onedayclass_topic",
    'whyBDA', 
    'what_to_gain',
]

In [61]:
df_new = df.drop(columns=columns_to_exclude, errors='ignore')

In [62]:
# NaN 비율 계산
nan_ratio = df_new.isna().sum() / len(df_new)

# 50% 이상인 열 목록
cols_to_drop = nan_ratio[nan_ratio > 0.5].index

# 50% 이상 비어있는 열 삭제
df_new.drop(columns=cols_to_drop, inplace=True)

In [63]:
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

# 1. 데이터 로드
# df_new는 이미 train 데이터로 로드된 상태라고 가정
test_df = pd.read_csv("test.csv")  # test.csv 따로 로드
test_df = test_df.drop(columns=columns_to_exclude, errors='ignore') # 자연어 열 삭제
test_df = test_df.drop(columns=cols_to_drop, errors='ignore') # 결측치 많은 열 삭제

# 2. 타겟 및 피처 분리
target_col = 'withdrawal'
id_col = 'ID'

X = df_new.drop(columns=[target_col, id_col])
y = df_new[target_col]
X_test_final = test_df.drop(columns=[id_col])

# 3. 범주형 변수 라벨 인코딩 (train+test 합쳐서!)
all_data = pd.concat([X, X_test_final], axis=0)
for col in all_data.select_dtypes(include=['object', 'category']).columns:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col].astype(str))

# 다시 나누기
X = all_data.iloc[:len(X), :]
X_test_final = all_data.iloc[len(X):, :]

# 4. train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. LightGBM 모델 학습
model = LGBMClassifier(random_state=42, is_unbalance=True)
model.fit(X_train, y_train)

# 6. 검증 성능 확인 (F1 Score)
val_pred = model.predict(X_val)
f1 = f1_score(y_val, val_pred)
print("F1 Score:", round(f1, 4))

# 7. test.csv 예측
test_pred = model.predict(X_test_final)

# 8. 제출 파일 생성
submission = pd.DataFrame({
    id_col: test_df[id_col],
    target_col: test_pred
})
submission.to_csv("submission.csv", index=False)
print("'submission.csv' 저장 완료!")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 583, number of negative: 261
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000557 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 252
[LightGBM] [Info] Number of data points in the train set: 844, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.690758 -> initscore=0.803667
[LightGBM] [Info] Start training from score 0.803667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -